In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib
stops = stopwords.words()

/home/takaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
features = [
#     'clean_q1_tokenized',
#     'clean_q2_tokenized',
#     'clean_q1_stem',
#     'clean_q2_stem',
#     'clean_q1_pos_tagged',
#     'clean_q2_pos_tagged',
    'clean_q1_lemmatized',
    'clean_q2_lemmatized',
#     'clean_q1_lemmatized_stem',
#     'clean_q2_lemmatized_stem'
]
def read_data(t, features):
    data = pd.read_csv('../input/{}.csv'.format(t))
    for feature in features:
        data = pd.merge(data, pd.read_pickle('../feature/{}_{}.pkl'.format(t, feature)))
    data.fillna(0.0)
    return data

In [3]:
train = read_data('train', features)#[:1000]
test = read_data('test', features)#[:1000]

In [4]:
lemm_words = train.clean_q1_lemmatized.tolist() + train.clean_q2_lemmatized.tolist() + test.clean_q1_lemmatized.tolist() + test.clean_q2_lemmatized.tolist()

In [5]:
from nltk import FreqDist
words = []
for s in lemm_words:
    words.extend(s)
freq = FreqDist(words)

In [6]:
sel_words = [w for w, f in freq.most_common()][:200]

In [ ]:
def extract(data, words):
    for i, row in data.iterrows():
        if i % 10000 == 0:
            print(i)

        q1_words = set(row['clean_q1_lemmatized'])
        q2_words = set(row['clean_q2_lemmatized'])
        for word in words:
            data.set_value(i, 'q1_{}'.format(word), word in q1_words)
            data.set_value(i, 'q2_{}'.format(word), word in q2_words)

    return data

In [ ]:
train = extract(train, sel_words)
test = extract(test, sel_words)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
0


In [ ]:
for feature in features:
    train[feature] = train[feature].astype(np.bool)
    test[feature] = test[feature].astype(np.bool)

In [ ]:
# import util
# util.save_feature(train, 'train', features, 'id')
# util.save_feature(test, 'test', features, 'test_id')

In [ ]:
for word in sel_words:
    train[word] = train['q1_' + word].astype(np.int8) + train['q2_' + word].astype(np.int8)
    test[word] = test['q1_' + word].astype(np.int8) + test['q2_' + word].astype(np.int8)

In [ ]:
import util
util.save_feature(train, 'train', sel_words, 'id')
util.save_feature(test, 'test', sel_words, 'test_id')